In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# SETTINGS

# Model Settings
random_seed=1
learning_rate=0.0001
batch_size=256
num_epochs=20

# Architecture
num_classes=10

# Other
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Dataset
train_indices=torch.arange(0, 48000)
valid_indices=torch.arange(48000, 50000)

train_transform=transforms.Compose([transforms.Resize((70, 70)),
                                    transforms.RandomCrop((64, 64)),
                                    transforms.ToTensor()])

test_transform=transforms.Compose([transforms.Resize((70, 70)),
                                   transforms.CenterCrop((64, 64)),
                                   transforms.ToTensor()])

train_and_valid=datasets.CIFAR10(root="data", train=True, transform=train_transform, download=True)

train_dataset=Subset(train_and_valid, train_indices)
valid_dataset=Subset(train_and_valid, valid_indices)
test_dataset=datasets.CIFAR10(root="data", train=False, transform=test_transform, download=False)

train_loader=DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=4)

valid_loader=DataLoader(dataset=valid_dataset, batch_size=batch_size, num_workers=4)

test_loader=DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=4)

Extracting data/cifar-10-python.tar.gz to data


In [4]:
# Checking the dataset
print("Training Set:\n")
for images, labels in train_loader:
    print("Image batch dimensions: ", images.shape)
    print("Image label dimensions: ", labels.shape)
    break

# Checking the dataset
print("\nValidation Set:")
for images, labels in valid_loader:
    print("Image batch dimensions: ", images.shape)
    print("Image label dimensions: ", labels.shape)
    break

# Checking the dataset
print("\nTest Set:")
for images, labels in test_loader:
    print("Image batch dimensions: ", images.shape)
    print("Image label dimensions: ", labels.shape)
    break

Training Set:



Image batch dimensions:  torch.Size([256, 3, 64, 64])
Image label dimensions:  torch.Size([256])


Validation Set:
Image batch dimensions:  torch.Size([256, 3, 64, 64])
Image label dimensions:  torch.Size([256])

Test Set:
Image batch dimensions:  torch.Size([256, 3, 64, 64])
Image label dimensions:  torch.Size([256])


In [5]:
# Model
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.features=nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
            )
        
        self.avgpool=nn.AdaptiveAvgPool2d((6, 6))
        
        self.classifier=nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256*6*6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
            )
    
    def forward(self, x):
        x=self.features(x)
        x=self.avgpool(x)
        x=x.view(x.size(0), 256*6*6)
        logits=self.classifier(x)
        probas=F.softmax(logits, dim=1)
        return logits, probas

In [6]:
torch.manual_seed(random_seed)
model=AlexNet(num_classes=num_classes)
model.to(device)

optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# Training
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples=0, 0
    model.eval()
    for i, (features, targets) in enumerate(data_loader):
        
        features=features.to(device)
        targets=targets.to(device)
        
        logits, probas=model(features)
        _, predicted_labels=torch.max(probas, 1)
        num_examples+=targets.size(0)
        assert predicted_labels.size()==targets.size()
        correct_pred+=(predicted_labels==targets).sum()
    return correct_pred.float()/num_examples*100


start_time=time.time()
cost_list=[]
train_acc_list, valid_acc_list=[], []

for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features=features.to(device)
        targets=targets.to(device)
        
        # Forward and backprop
        logits, probas=model(features)
        cost=F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        # Update model parameters
        optimizer.step()
        
        # ONLY FOR LOGGING
        cost_list.append(cost.item())
        if not batch_idx % 150:
            print("Batch: %03d/%03d | Epoch: %03d/%03d | Cost: %.2f" % (batch_idx, len(train_loader), epoch+1, num_epochs, cost))
        
    model.eval()
    with torch.set_grad_enabled(False):

        train_acc=compute_accuracy(model, train_loader, device)
        valid_acc=compute_accuracy(model, valid_loader, device)

        print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}\n'
              f'Train ACC: {train_acc:.2f} | Validation ACC: {valid_acc:.2f}')
        
        train_acc_list.append(train_acc)
        valid_acc_list.append(valid_acc)
    
    elapsed=(time.time()-start_time)/60
    print(f'Time Elapsed: {elapsed:.2f} min')

elapsed=(time.time()-start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')

Batch: 000/188 | Epoch: 001/020 | Cost: 2.30
Batch: 150/188 | Epoch: 001/020 | Cost: 1.74
Epoch: 001/020
Train ACC: 35.74 | Validation ACC: 37.15
Time Elapsed: 0.50 min
Batch: 000/188 | Epoch: 002/020 | Cost: 1.65
Batch: 150/188 | Epoch: 002/020 | Cost: 1.49
Epoch: 002/020
Train ACC: 44.17 | Validation ACC: 43.40
Time Elapsed: 1.01 min
Batch: 000/188 | Epoch: 003/020 | Cost: 1.48
Batch: 150/188 | Epoch: 003/020 | Cost: 1.30
Epoch: 003/020
Train ACC: 52.15 | Validation ACC: 53.15
Time Elapsed: 1.50 min
Batch: 000/188 | Epoch: 004/020 | Cost: 1.21
Batch: 150/188 | Epoch: 004/020 | Cost: 1.22
Epoch: 004/020
Train ACC: 56.81 | Validation ACC: 56.80
Time Elapsed: 2.01 min
Batch: 000/188 | Epoch: 005/020 | Cost: 1.13
Batch: 150/188 | Epoch: 005/020 | Cost: 1.14
Epoch: 005/020
Train ACC: 58.57 | Validation ACC: 59.20
Time Elapsed: 2.49 min
Batch: 000/188 | Epoch: 006/020 | Cost: 1.07
Batch: 150/188 | Epoch: 006/020 | Cost: 1.13
Epoch: 006/020
Train ACC: 61.35 | Validation ACC: 61.55
Time Elap

In [ ]:
model.eval()
with torch.set_grad_enabled(False): # save memory during inference
    test_acc=compute_accuracy(model=model, data_loader=test_loader, device=device)
    valid_acc=compute_accuracy(model=model, data_loader=valid_loader, device=device)

print("Valid Accuracy: %.2f%% || Test Accuracy: %.2f%%" % (valid_acc, test_acc))